In [4]:
from groq import Groq
from langchain_groq import ChatGroq
import os

from dotenv import load_dotenv
load_dotenv()


True

In [138]:
llm = ChatGroq(temperature=0,
                      model_name="llama-3.3-70b-specdec",
                      api_key=os.environ.get("GROQ_API_KEY"))

# llm.invoke("What is up")


In [139]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma


In [140]:
from langchain_cohere import CohereEmbeddings
embeddings = CohereEmbeddings(model="embed-english-v3.0")




In [141]:
import ssl
import certifi
from langchain.document_loaders import WebBaseLoader
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import requests

# Disable SSL warnings (use this with caution)
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# Create a custom SSL context that doesn't verify certificates
ssl_context = ssl.create_default_context(cafile=certifi.where())
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

# Use the custom SSL context with WebBaseLoader
urls = ['https://copyright.gov.in/Copyright_Rules_2013/chapter_i.html',
     'https://copyright.gov.in/Copyright_Rules_2013/chapter_ii.html', 
     'https://copyright.gov.in/Copyright_Rules_2013/chapter_iii.html', 
     'https://copyright.gov.in/Copyright_Rules_2013/chapter_iv.html', 
     'https://copyright.gov.in/Copyright_Rules_2013/chapter_v.html', 
     'https://copyright.gov.in/Copyright_Rules_2013/chapter_vi.html', 
     'https://copyright.gov.in/Copyright_Rules_2013/chapter_vii.html',
      'https://copyright.gov.in/Copyright_Rules_2013/chapter_viii.html',
       'https://copyright.gov.in/Copyright_Rules_2013/chapter_ix.html', 
       'https://copyright.gov.in/Copyright_Rules_2013/chapter_x.html', 
       'https://copyright.gov.in/Copyright_Rules_2013/chapter_xi.html', 
       'https://copyright.gov.in/Copyright_Rules_2013/chapter_xii.html', 
       'https://copyright.gov.in/Copyright_Rules_2013/chapter_xiii.html',
        'https://copyright.gov.in/Copyright_Rules_2013/chapter_xiv.html', 
        'https://copyright.gov.in/Copyright_Rules_2013/chapter_xv.html', 
        'https://copyright.gov.in/Copyright_Rules_2013/chapter_xvi.html', 
        'https://copyright.gov.in/Copyright_Rules_2013/chapter_xvii.html',
         'https://copyright.gov.in/Copyright_Rules_2013/chapter_xviii.html']

docs = []
for url in urls:
    loader = WebBaseLoader(url)
    loader.requests_kwargs = {'verify': False}  # Bypass SSL verification
    docs.append(loader.load())

docs_list = [item for sublist in docs for item in sublist]



In [142]:
# Split documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
split_docs = text_splitter.split_documents(docs_list)

# Clean up the page content by removing escape characters
for i, doc in enumerate(split_docs):
    # Replace common escape characters
    cleaned_text = doc.page_content.replace('\n', ' ').replace('\r', '').replace('\t', ' ')
    # Replace multiple spaces with a single space
    cleaned_text = ' '.join(cleaned_text.split())
    split_docs[i].page_content = cleaned_text

print(f"Number of documents after splitting: {len(split_docs)}")
if split_docs:
    print(f"Sample of cleaned text: {split_docs[0].page_content[:200]}...")

Number of documents after splitting: 352
Sample of cleaned text: CHAPTER I THE COPYRIGHT RULES, 2013 CHAPTER I CHAPTER II CHAPTER III CHAPTER IV CHAPTER V CHAPTER VI CHAPTER VII CHAPTER VIII CHAPTER IX CHAPTER X CHAPTER XI CHAPTER XII CHAPTER XIII CHAPTER XIV CHAPT...


In [ ]:
vectorstore = Chroma.from_documents(documents=split_docs,
                                    embedding=embeddings,
                                    collection_name="full-context",
                                    persist_directory="chroma.db")
                                    

In [144]:
retriever = vectorstore.as_retriever(search_kwargs={"k":7})


In [175]:
import time
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a 
    user question to a vectorstore or web search. Use the vectorstore for questions on copyright i.e Introduction to the Copyright Act, 1957, The Copyright (Amendment) Act, 2012, Reasons for Amendments, Digital Protection, Internet Service Provider (ISP) Liability, Statutory Licenses, Royalty Rights, Performer Rights, Copyright Societies, Exceptions for the Physically Disabled. You do not need to be stringent with the keywords 
    in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search' 
    or 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and 
    no premable or explaination. Question to route: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question"],
)

router_llm = ChatGroq(temperature=0,
                      model_name="llama3-8b-8192",
                      api_key=os.environ.get("GROQ_API_KEY"))
start = time.time()
question_router = prompt | router_llm | JsonOutputParser()
#
question = "What is copyright law?"
print(question_router.invoke({"question": question}))
end = time.time()
print(f"The time required to generate response by Router Chain in seconds:{end - start}")

{'datasource': 'vectorstore'}
The time required to generate response by Router Chain in seconds:1.3437728881835938


In [196]:
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    Try to give as much information as possible.
    c <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} 
    Context: {context} 
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
start = time.time()
rag_chain = prompt | llm | StrOutputParser()

In [ ]:
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance 
    of a retrieved document to a user question. If the document contains keywords related to the user question, 
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)


start = time.time()
retrieval_grader = prompt | llm | JsonOutputParser()
question = "Copyright laws and application"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))
end = time.time()
print(f"The time required to generate response by the retrieval grader in seconds:{end - start}")

{'score': 'yes'}
The time required to generate response by the retrieval grader in seconds:1.4636421203613281


In [179]:
prompt = PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether 
    an answer is grounded in / supported by a set of facts. Give a binary 'yes' or 'no' score to indicate 
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a 
    single key 'score' and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts:
    \n ------- \n
    {documents} 
    \n ------- \n
    Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "documents"],
)
start = time.time()
hallucination_grader = prompt | llm | JsonOutputParser()




In [180]:
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an 
    answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is 
    useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
    \n ------- \n
    {generation} 
    \n ------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "question"],
)
answer_grader = prompt | llm | JsonOutputParser()

In [232]:
prompt_intent = PromptTemplate(
    template="""
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a classifier assessing whether the query given is just asking a
    question based on the context or if it is giving a command. Give either 'command' or 'question_answering' to indicate whether
    the given query by the user. Provide the intent output in a single key 'intent' and no preamble or explanation.  <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the question/query:
    \n\n
    {question}
    \n\n
    """,
    input_variables=["question"],
)

intent_classifier = prompt_intent | llm | JsonOutputParser()

x = intent_classifier.invoke({"question": "Send a form for appealing copywrite for me"})


In [248]:
x

{'intent': 'command'}

In [233]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)


In [234]:
from typing_extensions import TypedDict
from typing import List, Dict, Any
### State

class GraphState(TypedDict):
    question : str
    generation : str
    web_search : str
    documents : List[str] = []
    context: int # this is the user id
    user_data: Dict[str, Any]
    form_struct: Dict[str, Any]
    

In [235]:
from langchain.schema import Document
def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}
#
def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    question = state["question"]
    documents = state["documents"]
    
    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}
#
def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]
    
    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score['score']
        # Document relevant
        if grade.lower() == "yes":
             filtered_docs.append(d)
        # Document not relevant
        else:
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}
#
def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    question = state["question"]
    # Initialize documents as empty list if it doesn't exist
    documents = state.get("documents", [])

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    if documents:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "question": question}

In [236]:
def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """


    question = state["question"]
    print(question)
    source = question_router.invoke({"question": question})  
    print(source)
    print(source['datasource'])
    if source['datasource'] == 'web_search':
         return "websearch"
    elif source['datasource'] == 'vectorstore':
        return "vectorstore"

In [237]:
def route_intent(state):
    question = state["question"]
    classification = intent_classifier.invoke({"question": question})
    return classification

In [238]:
def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """


    question = state["question"]
    web_search = state["web_search"]
    filtered_documents = state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query

        return "websearch"
    else:
        # We have relevant documents, so generate answer

        return "generate"

In [239]:
def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    grade = score['score']

    # Check hallucination
    if grade == "yes":
        score = answer_grader.invoke({"question": question,"generation": generation})
        grade = score['score']
        if grade == "yes":
            return "useful"
        else:
            return "not useful"
    else:
        return "not supported"

In [240]:
def entry_data(state):
    """
    This node doesn't modify the state - it simply passes it through without changes.
    This can be useful for debugging or as a placeholder in the workflow.
    
    Args:
        state (dict): The current graph state
        
    Returns:
        state (dict): The unchanged state
    """
    print("This goes as an entry for parallel action of getting user data and form data")
    return state

In [241]:
def user_data_sql(state):
    # Get user data
    user_data = {}
    return {"user_data": user_data}
    

In [242]:
def get_form_struct(state):
    # form data from RAG

    form_struct = {}
    return {"form_struct": form_struct}

In [243]:
def merge_node(state):
    # Takes keys from user_data and adds it to form struct
    new_form_stuct = {
        "name": "sidhant"
    }
    return {"form_struct": new_form_stuct}

In [244]:
from langgraph.graph import END, StateGraph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search) # web search
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("grade_documents", grade_documents) # grade documents
workflow.add_node("generate", generate) 
workflow.add_node("entry_data", entry_data)
workflow.add_node("user_data_sql", user_data_sql)
workflow.add_node("get_form_struct", get_form_struct)
workflow.add_node("merge_node", merge_node)
workflow.add_node("route_intent", route_intent)


In [245]:
workflow.set_conditional_entry_point(
    route_question,
    {
        "websearch": "websearch",
        "vectorstore": "retrieve",
    },
)



workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)
workflow.add_edge("websearch", "generate")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": "route_intent",  # Now this points to a valid node
        "not useful": "websearch",
    },
)

workflow.add_conditional_edges(
    "route_intent",
    lambda x: x["intent"],  # Use the output of the route_intent function
    {
        "question_answering": END,
        "command": "entry_data"
    }
)
workflow.add_edge("entry_data", "user_data_sql")
workflow.add_edge("entry_data", "get_form_struct")
workflow.add_edge("user_data_sql", "merge_node")
workflow.add_edge("get_form_struct", "merge_node")
workflow.add_edge("merge_node", END)

In [246]:
app = workflow.compile()
